<a href="https://colab.research.google.com/github/DANCAR1969/programacion/blob/master/Proyecto_Final_IAutoBody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 Proyecto Final

Importar las bibliotecas
# Importamos las bibliotecas necesarias
import pandas as pd # manipulación y análisis de datos
import numpy as np # cálculos numéricos y manejo de matrices
import matplotlib.pyplot as plt # generar gráficos
import seaborn as sns # visualización de datos con mejores gráficos
import spacy # tareas de NLP
from sklearn.neighbors import KNeighborsRegressor # modelo K-Nearest Neighbors para regresión
from sklearn.preprocessing import StandardScaler # para normalizar los datos
from google.colab import drive # para conectar Google Drive en Google Colab
drive.mount('/content/drive')
# Instalar spaCy para el procesamiento NLP
!pip install spacy
# Descargar el modelo de lenguaje español
!python -m spacy download es_core_news_sm
Mostrar el resultado oculto
# Cargar el archivo Excel
df = pd.read_excel("/content/drive/MyDrive/lista materiales cs/LISTA MATERIALES.xlsx")
df.head()
print(df.info())
Mostrar el resultado oculto
Limpieza y organización de los datos existentes
# Convertir la columna 'ITEM' a tipo númerico
# los errores se convierten en NaN, luego se rellenan con 0 y finalmente se transforman en enteros
df['ITEM'] = pd.to_numeric(df['ITEM'], errors='coerce').fillna(0).astype(int)
# Convertir la columna referencia a un indice númerico
df['OP'] = pd.factorize(df['OP'])[0] + 1 # Asigna un valor numérico único a cada valor distinto en la columna 'OP'
# Normalizar texto en columna NIVEL 1
# Se estandariza el texto eliminando espacios innecesarios y colocando en mayúsculas
def normalize_text(text):
if isinstance(text, str): # Verifica si el valor es texto
return text.strip().upper() # Elimina espacios en blanco al inicio y final, y convierte el texto a mayúsculas
return text # Si no es texto, lo deja igual
df['NIVEL 1'] = df['NIVEL 1'].apply(normalize_text)
# Ajustar la columna de unidades
df['UNIDAD'] = df['UNIDAD'].replace({
'UND': 'U', 'MTS': 'm', 'GLS': 'Gal', 'CUA': 'Cuarto',
'KIL': 'Kg', 'MT2': 'm2', 'LTS': 'Lt', '#N/A': '0'
})
# Filtrar ítems que tengan al menos 10 muestras
item_counts = df['ITEM'].value_counts() # Cuenta la cantidad de veces que aparece cada 'ITEM'
df = df[df['ITEM'].isin(item_counts[item_counts >= 10].index)] # Filtra los 'ITEM' que aparecen al menos 10 veces
# Verificar datos nulos para asegurar que la limpieza fue exitosa
print("Valores nulos por columna después de la limpieza:")
print(df.isnull().sum()) # Muestra la cantidad de valores nulos en cada columna después del proceso
# Mostrar las primeras filas para revisar el resultado
print("\nPrimeras filas de la tabla limpia:")
print(df.head())
print(df.info())
Datos_organizados = df
20/3/25, 7:22 p.m. PROYECTO FINAL jueves.ipynb - Colab

https://colab.research.google.com/drive/17M25Ey4-LJHJOiob2Sccvm3mimd-jZFg#printMode=true 1/9

Datos_organizados.to_excel('/content/drive/MyDrive/lista materiales cs/LISTA MATERIALES.xlsx', index=False)
print(f"Datos organizados CNN1 en Excel.")
Mostrar el resultado oculto
REVISAMOS LA DISTRIBUCIÓN DE LOS DATOS
# Histogramas
plt.figure(figsize=(15, 5))
for i, col in enumerate(['LARGO EXT (mm)', 'ANCHO EXT (mm)', 'ALTO INTERNO (mm)'], 1): # Itera sobre las tres columnas indicadas
plt.subplot(1, 3, i) # Crea subdivisión de 1 fila y 3 columnas, pone cada gráfico en la columna correspondiente
sns.histplot(df[col], kde=True, bins=30) # Genera histograma con 30 bins y una curva de densidad (kde)
plt.title(f"Distribución del {col}") # Título del gráfico
plt.xlabel(f"{col}") # Etiqueta del eje x
plt.ylabel("Frecuencia") # Etiqueta del eje y
plt.tight_layout() # Ajusta automáticamente los elementos del gráfico para evitar que se superpongan
plt.show()
# Diagramas de caja
plt.figure(figsize=(15, 5))
for i, col in enumerate(['LARGO EXT (mm)', 'ANCHO EXT (mm)', 'ALTO INTERNO (mm)'], 1): # Itera sobre las mismas columnas
plt.subplot(1, 3, i) # Crea subdivisión de 1 fila y 3 columnas, pone cada gráfico en la columna correspondiente
sns.boxplot(x=df[col], whis=1.5) # Genera un diagrama de caja con un rango de bigotes (whis) del 1.5 veces el IQR (rango intercuartil)
plt.title(f"Diagrama de Cajas - {col}")
plt.xlabel(f"{col}")
plt.tight_layout()
plt.show()

Generamos datos sinteticos
20/3/25, 7:22 p.m. PROYECTO FINAL jueves.ipynb - Colab

https://colab.research.google.com/drive/17M25Ey4-LJHJOiob2Sccvm3mimd-jZFg#printMode=true 2/9

# Función para generar datos sintéticos
def generar_datos_sinteticos(data, n_sinteticos=10000): # Definimos la función
np.random.seed(42) # Semilla para reproducibilidad. Asegura que los datos sean siempre los mismos.
# Listas para almacenar los datos sintéticos
datos_sinteticos = [] # lista vacía
# Rango de dimensiones basado en las condiciones dadas
largo_min, largo_max = 2500, 8200
ancho_min, ancho_max = 1700, 2600
alto_min, alto_max = 1400, 3000
# Generar datos sintéticos
for _ in range(n_sinteticos): # Itera hasta generar la cantidad deseada de datos sintéticos
largo = np.random.randint(largo_min, largo_max + 1) # Genera un valor entero aleatorio dentro del rango de largo
ancho = np.random.randint(ancho_min, ancho_max + 1) # Genera un valor entero aleatorio dentro del rango de ancho
# Generar alto con relación lógica al ancho para evitar carrocerías muy esbeltas
if ancho < 2000:
alto = np.random.randint(1400, min(2200, 3000) + 1) # Si el ancho es menor a 2000 mm, el alto se limita a máximo 2200 mm
elif ancho < 2400:
alto = np.random.randint(2000, min(2400, 3000) + 1) # Si el ancho está entre 2000 y 2400 mm, el alto se limita a 2400 mm
else:
alto = np.random.randint(2200, 3000 + 1) # Si el ancho es mayor o igual a 2400 mm, el alto llega hasta el máximo de 3000 mm
# Selección aleatoria de un ítem existente
item = np.random.choice(data['ITEM'].unique()) # Selecciona aleatoriamente un ítem único del conjunto de datos original
# Determinar unidad y generar cantidad acorde
unidad = data[data['ITEM'] == item]['UNIDAD'].iloc[0] # Se obtiene la unidad del ítem seleccionado. iloc selecciona el 1er valor
if unidad == 'U':
cantidad = np.random.randint(1, 51) # Si la unidad es 'U', genera una cantidad entera entre 1 y 50
else:
cantidad = round(np.random.uniform(0.5, 20), 2) # Si no es 'U', genera una cantidad decimal entre 0.5 y 20
# Agregar fila sintética
datos_sinteticos.append([ # Se genera un identificador para la nueva fila sintética
f"OP_SINT_{_+1}", largo, ancho, alto, # Se agregan las dimensiones generadas
data[data['ITEM'] == item]['NIVEL 1'].iloc[0], # Se obtiene el valor de 'NIVEL 1' del ítem seleccionado
item,
data[data['ITEM'] == item]['MATERIAL'].iloc[0],
cantidad,
unidad
])
# Crear DataFrame con los datos generados
columnas = data.columns # Se obtiene la lista de columnas del DataFrame original
datos_sinteticos_df = pd.DataFrame(datos_sinteticos, columns=columnas) # Se crea el DataFrame con los datos sintéticos
return datos_sinteticos_df
# Generar datos sintéticos
datos_sinteticos = generar_datos_sinteticos(df)
# Combinar los datos originales con los datos sintéticos
datos_combinados = pd.concat([df, datos_sinteticos], ignore_index=True) # Concatena ambos df
# Exportar el resultado a un nuevo archivo Excel
datos_combinados.to_excel("/content/drive/MyDrive/lista materiales cs/LISTA MATERIALES.xlsx", index=False)
print("✅ Datos sintéticos generados y exportados correctamente.")
✅ Datos sintéticos generados y exportados correctamente.
Código ajustado en columna unidades para entregar la predicción
# Cargar el archivo Excel
df = pd.read_excel("/content/drive/MyDrive/lista materiales cs/LISTA MATERIALES.xlsx")
# Renombrar columnas para mayor claridad
df.columns = ['OP', 'LARGO_EXT', 'ANCHO_EXT', 'ALTO_INTERNO', 'NIVEL', 'ITEM', 'MATERIAL', 'CANTIDAD', 'UNIDAD']
# Limpiar y convertir columnas numéricas
20/3/25, 7:22 p.m. PROYECTO FINAL jueves.ipynb - Colab

https://colab.research.google.com/drive/17M25Ey4-LJHJOiob2Sccvm3mimd-jZFg#printMode=true 3/9

df['LARGO_EXT'] = pd.to_numeric(df['LARGO_EXT'], errors='coerce')
df['ANCHO_EXT'] = pd.to_numeric(df['ANCHO_EXT'], errors='coerce')
df['ALTO_INTERNO'] = pd.to_numeric(df['ALTO_INTERNO'], errors='coerce')
df['CANTIDAD'] = pd.to_numeric(df['CANTIDAD'], errors='coerce')
# Eliminar filas con valores faltantes en columnas clave
df = df.dropna(subset=['LARGO_EXT', 'ANCHO_EXT', 'ALTO_INTERNO', 'ITEM', 'CANTIDAD'])
# Crear diccionario de items para nombres y unidades
items_dict = {} # diccionario vacío para información clave de cada ITEM.
unique_items = sorted(df['ITEM'].unique()) # lista de todos los valores únicos en la columna ITEM, ordenados de menor a mayor.
for item in unique_items:
item_rows = df[df['ITEM'] == item] # filtra el df para obtener solo las filas que contienen el ITEM actual en el bucle.
if not item_rows.empty: # Esta condición evita que se intente acceder a un DataFrame vacío
items_dict[item] = { # se crea una entrada en el diccionario
'name': item_rows['MATERIAL'].iloc[0],
'unit': item_rows['UNIDAD'].iloc[0]
}
# Entrenar modelos para cada item
X_cols = ['LARGO_EXT', 'ANCHO_EXT', 'ALTO_INTERNO'] # variables de entrada
scaler = StandardScaler()
model_dict = {} # Diccionario vacío donde se guardarán los modelos entrenados para cada ITEM
for item in unique_items: # Filtrar las filas del DataFrame que correspondan al ITEM actual
item_data = df[df['ITEM'] == item]
if len(item_data) < 3: # Si hay menos de 3, se omite (evita problemas en el entrenamiento)
continue
# Extraer las columnas de entrada (largo, ancho y alto) como matriz de datos (X)
X = item_data[X_cols].values # `.values` convierte el DataFrame en un array de NumPy para el modelo
# Extraer la columna 'CANTIDAD' como vector objetivo (y)
y = item_data['CANTIDAD'].values
# Normalizar los datos de entrada para que todas las variables estén en la misma escala
X_scaled = scaler.fit_transform(X)
# Determinar el número óptimo de vecinos para el modelo KNN
# Se selecciona el mínimo entre 5 y la mitad del total de datos, asegurando al menos 1 vecino
k = min(5, max(1, len(item_data) // 2))
# Crear el modelo KNN con el número de vecinos determinado y asignar pesos según la distancia
model = KNeighborsRegressor(n_neighbors=k, weights='distance')
model.fit(X_scaled, y) # Entrena el modelo con los datos normalizados y el vector objetivo
model_dict[item] = model # Guardar el modelo entrenado en el diccionario usando el ITEM como clave
# --- Ingreso de datos para predicción ---
print("Ingrese las dimensiones de la carrocería:")
length = float(input("Largo (mm): "))
width = float(input("Ancho (mm): "))
height = float(input("Alto (mm): "))
# Vector de entrada escalado
X_input = np.array([[length, width, height]])
X_input_scaled = scaler.transform(X_input)
# Palabras clave para cantidades en números enteros
elementos_enteros = ["TORNILLO", "TUERCA", "WASA", "ARANDELA", "BISAGRA", "TERMINAL", "SWITCH",
"LAMPARA", "BROCA", "FUSIBLE", "BREAKER", "PORTA RELAY", "LIJA", "DISCO VELCRO",
"PANOS", "AMARRA", "PUBLICIDAD", "REMACHE", "SOUDAFLEX", "CINTA", "REGLETA", "INMOVILIZADOR", "DISCO", "SIKA FLEX", "VA

# Predicción de materiales
results = []
for item, model in model_dict.items(): # Iterar sobre cada item y su modelo en el diccionario de modelos
predicted_qty = max(0, model.predict(X_input_scaled)[0]) # Se toma el primer valor de la predicción y se asegura que no sea negativo (se
if predicted_qty > 0.1: # Si la cantidad es mayor a 0.1, se considera válida
# Obtener el nombre del ítem en mayúsculas y su unidad de medida
item_name = items_dict[item]['name'].upper()
unit = items_dict[item]['unit']
# Verificar si el nombre del ítem coincide con algún elemento en 'elementos_enteros'
# Si es así, se redondea a un número entero (evita decimales en elementos como tornillos o arandelas)
if any(elemento in item_name for elemento in elementos_enteros):
predicted_qty = round(predicted_qty)
# Si la unidad de medida es 'UND', también se redondea a entero
elif unit == 'UND':
predicted_qty = round(predicted_qty)

20/3/25, 7:22 p.m. PROYECTO FINAL jueves.ipynb - Colab

https://colab.research.google.com/drive/17M25Ey4-LJHJOiob2Sccvm3mimd-jZFg#printMode=true 4/9

# Si no cumple ninguna de las condiciones anteriores, se redondea a 2 decimales
else:
predicted_qty = round(predicted_qty, 2)
# Si la cantidad predicha es mayor que cero, se agrega a la lista de resultados
if predicted_qty > 0:
results.append({
'ITEM': item,
'MATERIAL': items_dict[item]['name'],
'CANTIDAD REQUERIDA': predicted_qty,
'UNIDAD': unit
})
# Crear DataFrame con resultados
results_df = pd.DataFrame(results)
# Exportar a Excel
output_path = "/content/drive/MyDrive/lista materiales cs/Resultados_Prediccion_Claude 1.xlsx"
results_df.to_excel (output_path, index=False)
print(f"Resultados exportados a {output_path}")

Ingrese las dimensiones de la carrocería:
Largo (mm): 3400
Ancho (mm): 2000
Alto (mm): 1780
Resultados exportados a /content/drive/MyDrive/lista materiales cs/Resultados_Prediccion_Claude 1.xlsx

Realizar un análisis comparativo entre los materiales en stock y la predicción de materiales para la fabricacion de un furgon carga seca.
 Comparación de datos

# Cargar las hojas de cálculo en DataFrames de Pandas
stock_df = pd.read_excel("/content/drive/MyDrive/Stock/STOCK DE MATERIALES.xlsx")
prediccion_df = pd.read_excel("/content/drive/MyDrive/lista materiales cs/Resultados_Prediccion_Claude 1.xlsx")
# Diccionario de abreviaciones para las unidades
abreviaturas = {
'UND': 'U',
'MTS': 'm',
'GLS': 'Gal',
'CUA': 'Cuarto',
'KIL': 'Kg',
'MT2': 'm2',
'LTS': 'Lt',
'#N/A': '0'
}
# Reemplazar las unidades en la columna 'UNIDAD' según el diccionario
stock_df['UNIDAD'] = stock_df['UNIDAD'].map(abreviaturas).fillna(stock_df['UNIDAD'])
# Convertir las columnas a tipo numérico (en caso de que haya valores no numéricos)
stock_df['CANTIDAD EN STOCK'] = pd.to_numeric(stock_df['CANTIDAD EN STOCK'], errors='coerce')
prediccion_df['CANTIDAD REQUERIDA'] = pd.to_numeric(prediccion_df['CANTIDAD REQUERIDA'], errors='coerce')
# Reemplazar valores nulos en la columna 'CANTIDAD EN STOCK' por un valor predeterminado (por ejemplo, 0)
stock_df['CANTIDAD EN STOCK'] = stock_df['CANTIDAD EN STOCK'].fillna(0)
# Función para comparar el stock con la predicción
def comparar_stock_con_prediccion(stock_df, prediccion_df):
# Asegurarse de que los nombres de las columnas coincidan (eliminando espacios extra)
stock_df.columns = stock_df.columns.str.strip()
prediccion_df.columns = prediccion_df.columns.str.strip()
# Realizar un merge(Union) entre las dos hojas por la columna "MATERIAL"
comparacion_df = pd.merge(prediccion_df, stock_df, how='left', on='MATERIAL')
# Añadir una columna para comparar si hay suficiente stock
comparacion_df['Suficiente Stock'] = comparacion_df['CANTIDAD EN STOCK'] >= comparacion_df['CANTIDAD REQUERIDA']
# Añadir una columna para la cantidad faltante (si la hay)
comparacion_df['Cantidad Faltante'] = comparacion_df['CANTIDAD REQUERIDA'] - comparacion_df['CANTIDAD EN STOCK']
comparacion_df['Cantidad Faltante'] = comparacion_df['Cantidad Faltante'].apply(lambda x: x if x > 0 else 0)
20/3/25, 7:22 p.m. PROYECTO FINAL jueves.ipynb - Colab

https://colab.research.google.com/drive/17M25Ey4-LJHJOiob2Sccvm3mimd-jZFg#printMode=true 5/9

# Crear una columna de "Estado" que diga si el material está disponible o no
comparacion_df['Estado'] = comparacion_df['Suficiente Stock'].apply(lambda x: "Suficiente stock" if x else "No hay suficiente stock")
return comparacion_df
# Llamar a la función para comparar stock con predicción
resultado_comparacion = comparar_stock_con_prediccion(stock_df, prediccion_df)
# Asegurarse de que la comparación se realizó correctamente
if resultado_comparacion is not None and not resultado_comparacion.empty:
# Mostrar el resultado
print("\nResultado de la comparación:")
print(resultado_comparacion)
# Guardar el resultado en un archivo Excel
resultado_comparacion.to_excel("/content/drive/MyDrive/resultado_comparacion.xlsx", index=False)
print("El archivo se ha guardado correctamente.")
else:
print("No se pudo generar el archivo debido a que la comparación no produjo resultados válidos.")
Mostrar el resultado oculto
El resultado es un análisis que muestra si hay stock su

ciente para satisfacer la demanda predicha y calcular cualquier falta de material.

Indicar qué materiales faltan en el stock para la fabricacion del furgon y cómo gestionar el pedido.
 Análisis de datos y NLP(Procesamiento de lenguaje Natural)

# Cargar el archivo Excel en un DataFrame
stock_df = pd.read_excel("/content/drive/MyDrive/resultado_comparacion.xlsx")
# Diccionario de abreviaturas para las unidades de medición
abreviaturas = {
'UND': 'U',
'MTS': 'm',
'GLS': 'Gal',
'CUA': 'Cuarto',
'KIL': 'Kg',
'MT2': 'm2',
'LTS': 'Lt',
'#N/A': '0'
}
# Función para generar un mensaje con los materiales faltantes y su unidad de medición
def generar_mensaje_nlp(comparacion_df):
mensaje = "Se requiere solicitar los siguientes materiales debido a que el stock es insuficiente:\n\n"
# Filtrar los materiales que faltan (solo aquellos con 'Cantidad Faltante' mayor a 0)
materiales_faltantes = comparacion_df[comparacion_df['Cantidad Faltante'] > 0]
# Repite los materiales faltantes y generar el mensaje
for index, row in materiales_faltantes.iterrows():
material = row['MATERIAL']
cantidad_faltante = row['Cantidad Faltante']
unidad = row['UNIDAD_x'] # Usamos la columna de unidad correspondiente
# Obtener la unidad abreviada usando el diccionario
unidad_abreviada = abreviaturas.get(unidad, unidad) # Si no está en el diccionario, se usa el valor original
# Agregar al mensaje el material, cantidad faltante y la unidad de medición
mensaje += f"- {material}: {cantidad_faltante} {unidad_abreviada}\n"
mensaje += "\nFavor de gestionar el pedido lo antes posible."
return mensaje
# Llamar a la función para generar el mensaje
mensaje_generado = generar_mensaje_nlp(stock_df)
# Imprimir el mensaje generado
print(mensaje_generado)
Mostrar el resultado oculto
20/3/25, 7:22 p.m. PROYECTO FINAL jueves.ipynb - Colab

https://colab.research.google.com/drive/17M25Ey4-LJHJOiob2Sccvm3mimd-jZFg#printMode=true 6/9

 GAN Condicional
 Generación de imágenes sintéticas
import cv2
import numpy as np
import os
import random
def generate_synthetic_image(output_path, width, height, vertical_bars, horizontal_bars):
image = np.ones((4000, 4000, 3), dtype=np.uint8) * 255 # Fondo blanco
x1 = (4000 - width) // 2 # Centrar rectángulo en el lienzo
y1 = (4000 - height) // 2
# Dibuja el rectángulo grande con tamaño dinámico
border_thickness = 3
cv2.rectangle(image, (x1, y1), (x1 + width, y1 + height), (0, 0, 0), border_thickness)
# Tamaño fijo de las barras
bar_height = 80 # Altura de barras horizontales
bar_width = 80 # Ancho de barras verticales
# Espacio disponible para distribuir barras
available_height = height - 2 * bar_height # Espacio sin contar las barras superior e inferior
available_width = width - 2 * bar_width # Espacio sin contar las barras de los extremos
# Distribuir barras horizontales (siempre 2 en los extremos, resto equidistantes)
for i in range(horizontal_bars):
if i == 0: # Primera barra (arriba)
y = y1
elif i == horizontal_bars - 1: # Última barra (abajo)
y = y1 + height - bar_height
else: # Barras intermedias
y = y1 + bar_height + (i - 1) * (available_height // (horizontal_bars - 2))
cv2.rectangle(image, (x1, y), (x1 + width, y + bar_height), (255, 0, 0), -1)
# Distribuir barras verticales (siempre 2 en los extremos, resto equidistantes)
for i in range(vertical_bars):
if i == 0: # Primera barra (izquierda)
x = x1
elif i == vertical_bars - 1: # Última barra (derecha)
x = x1 + width - bar_width
else: # Barras intermedias
x = x1 + bar_width + (i - 1) * (available_width // (vertical_bars - 2))
cv2.rectangle(image, (x, y1), (x + bar_width, y1 + height), (0, 0, 255), -1)
# Guardar la imagen generada
cv2.imwrite(output_path, image)
# Crear carpeta para guardar imágenes
output_folder = "imagenessinteticas"
os.makedirs(output_folder, exist_ok=True)
for i in range(3000):
width = random.randint(1400, 2600)
height = random.randint(1400, 2800)
# Definir el número mínimo y máximo de barras basado en el tamaño
max_horizontal_bars = min(10, (height - 2 * 80) // 100 + 2) # Mínimo 2, máximo ajustado al espacio
max_vertical_bars = min(10, (width - 2 * 80) // 100 + 2) # Mínimo 2, máximo ajustado al espacio
horizontal_bars = random.randint(2, max_horizontal_bars)
vertical_bars = random.randint(2, max_vertical_bars)
output_path = os.path.join(output_folder, f"image_{i+1}.png")
generate_synthetic_image(output_path, width, height, vertical_bars, horizontal_bars)
20/3/25, 7:22 p.m. PROYECTO FINAL jueves.ipynb - Colab

https://colab.research.google.com/drive/17M25Ey4-LJHJOiob2Sccvm3mimd-jZFg#printMode=true 7/9

 GAN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
# Parámetros
IMG_SHAPE = (64, 64, 1) # Tamaño de las imágenes
NOISE_DIM = 100 # Dimensión del ruido
BATCH_SIZE = 32
EPOCHS = 5000
# Rango de dimensiones de los rectángulos
WIDTH_RANGE = (10, 50) # Ancho mínimo y máximo
HEIGHT_RANGE = (10, 50) # Alto mínimo y máximo
# Cargar dataset de imágenes de rectángulos
def load_data():
data = []
folder = "/content/imagenessinteticas"
for file in os.listdir(folder):
img = tf.keras.preprocessing.image.load_img(os.path.join(folder, file), color_mode="grayscale", target_size=IMG_SHAPE[:2])
img = np.array(img) / 127.5 - 1 # Normalización a [-1, 1]
data.append(img)
return np.expand_dims(np.array(data), axis=-1)
# Generador modificado para incluir width y height
def build_generator():
input_noise = keras.Input(shape=(NOISE_DIM,))
input_width = keras.Input(shape=(1,))
input_height = keras.Input(shape=(1,))
# Concatenar ruido con dimensiones normalizadas
merged_input = layers.Concatenate()([input_noise, input_width, input_height])
x = layers.Dense(8 * 8 * 128, activation="relu")(merged_input)
x = layers.Reshape((8, 8, 128))(x)
x = layers.Conv2DTranspose(128, kernel_size=7, strides=2, padding="same", activation='relu')(x)
x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation='relu')(x)
x = layers.Conv2DTranspose(1, kernel_size=4, strides=2, padding="same", activation='tanh')(x)
return keras.Model([input_noise, input_width, input_height], x)
# Discriminador (sin cambios)
def build_discriminator():
model = keras.Sequential([
layers.Conv2D(64, kernel_size=7, strides=2, padding="same", input_shape=IMG_SHAPE),
layers.LeakyReLU(alpha=0.2),
layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
layers.LeakyReLU(alpha=0.2),
layers.Flatten(),
layers.Dense(1, activation='sigmoid')
])
return model
# Construcción de la GAN
def build_gan(generator, discriminator):
discriminator.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])
discriminator.trainable = False
noise_input = keras.Input(shape=(NOISE_DIM,))
width_input = keras.Input(shape=(1,))
height_input = keras.Input(shape=(1,))
fake_img = generator([noise_input, width_input, height_input])
gan_output = discriminator(fake_img)
gan = keras.Model([noise_input, width_input, height_input], gan_output)
gan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.0002, 0.5))
return gan
# Entrenar GAN
20/3/25, 7:22 p.m. PROYECTO FINAL jueves.ipynb - Colab

https://colab.research.google.com/drive/17M25Ey4-LJHJOiob2Sccvm3mimd-jZFg#printMode=true 8/9

def train_gan(generator, discriminator, gan, dataset, epochs=EPOCHS, batch_size=BATCH_SIZE):
half_batch = batch_size // 2
for epoch in range(epochs):
# Entrenar el discriminador con imágenes reales y falsas
idx = np.random.randint(0, dataset.shape[0], half_batch)
real_imgs = dataset[idx]
# Generar ruido y dimensiones aleatorias
noise = np.random.normal(0, 1, (half_batch, NOISE_DIM))
widths = np.random.uniform(WIDTH_RANGE[0], WIDTH_RANGE[1], (half_batch, 1)) / IMG_SHAPE[1]
heights = np.random.uniform(HEIGHT_RANGE[0], HEIGHT_RANGE[1], (half_batch, 1)) / IMG_SHAPE[0]
fake_imgs = generator.predict([noise, widths, heights])
d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((half_batch, 1)))
d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
# Entrenar el generador para engañar al discriminador
noise = np.random.normal(0, 1, (batch_size, NOISE_DIM))
widths = np.random.uniform(WIDTH_RANGE[0], WIDTH_RANGE[1], (batch_size, 1)) / IMG_SHAPE[1]
heights = np.random.uniform(HEIGHT_RANGE[0], HEIGHT_RANGE[1], (batch_size, 1)) / IMG_SHAPE[0]
g_loss = gan.train_on_batch([noise, widths, heights], np.ones((batch_size, 1)))
if epoch % 500 == 0:
print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
save_generated_image(generator, epoch)
# Guardar imágenes generadas
def save_generated_image(generator, epoch):
noise = np.random.normal(0, 1, (1, NOISE_DIM))
width = np.array([[np.random.uniform(WIDTH_RANGE[0], WIDTH_RANGE[1]) / IMG_SHAPE[1]]])
height = np.array([[np.random.uniform(HEIGHT_RANGE[0], HEIGHT_RANGE[1]) / IMG_SHAPE[0]]])
generated_img = generator.predict([noise, width, height])[0]
generated_img = (generated_img + 1) / 2 # Desnormalizar a [0, 1]
plt.imshow(generated_img.squeeze(), cmap='gray')
plt.axis('off')
plt.savefig(f"generated/epoch_{epoch}.png")
plt.close()
# Ejecutar entrenamiento
if __name__ == "__main__":
dataset = load_data()
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)
os.makedirs("generated", exist_ok=True)
train_gan(generator, discriminator, gan, dataset)